In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
%cd /content

# 切到你的專案資料夾
%cd /content/drive/MyDrive/LSTM_PROGRAM


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
/content/drive/MyDrive/LSTM_PROGRAM


In [8]:
import os
import pandas as pd
import re
from functools import reduce

# 2. 【用戶自定義區間】
start_date = '2022/04/28'
end_date = '2025/07/01'
start = pd.to_datetime(start_date)
end = pd.to_datetime(end_date)

# 路徑和檔名
folder = '.'
csv_files_day = [
    './source_data/BTCUSDT_DAY.csv',
    './source_data/ETHUSDT_DAY.csv',
    './source_data/CME_MINI_DL_ES1!, 1D.csv',
    './source_data/CME_MINI_DL_NQ1!, 1D.csv',
    './source_data/CBOT_MINI_DL_YM1!, 1D.csv',
    './source_data/CBOE_DLY_VX1!, 1D.csv',
    './source_data/CBOT_DL_ZN1!, 1D.csv',
    './source_data/CBOT_DL_ZT1!, 1D.csv',
    './source_data/CBOT_DL_ZF1!, 1D.csv',
    './source_data/CBOT_DL_TN1!, 1D.csv',
    './source_data/CBOT_DL_ZB1!, 1D.csv',
    './source_data/CME_DL_SR31!, 1D.csv',
#    './source_data/TVC_US10Y, 1D.csv',
    './source_data/COMEX_DL_GC1!, 1D.csv',
#    './source_data/BITSTAMP_DAIUSD, 1D.csv',
    './source_data/BITSTAMP_USDCUSD, 1D.csv',
    './source_data/BITSTAMP_USDTUSD, 1D.csv'
#    './source_data/TAIFEX_DLY_TXF1!, 1D.csv'
]

csv_files_hour = [
    './source_data/BTCUSDT_HOUR.csv',
    './source_data/ETHUSDT_HOUR.csv',
    './source_data/CME_MINI_DL_ES1!, 60.csv',
    './source_data/CME_MINI_DL_NQ1!, 60.csv',
    './source_data/CBOT_MINI_DL_YM1!, 60.csv',
    './source_data/CBOE_DLY_VX1!, 60.csv',
    './source_data/CBOT_DL_ZN1!, 60.csv',
    './source_data/CBOT_DL_ZT1!, 60.csv',
    './source_data/CBOT_DL_ZF1!, 60.csv',
    './source_data/CBOT_DL_ZB1!, 60.csv',
    './source_data/CBOT_DL_TN1!, 60.csv',
    './source_data/CME_DL_SR31!, 60.csv',
#    './source_data/TVC_US10Y, 60.csv',
    './source_data/COMEX_DL_GC1!, 60.csv',
#    './source_data/BITSTAMP_DAIUSD, 60.csv',
    './source_data/Bitstamp_USDCUSD_1h_cleaned.csv',
    './source_data/Bitstamp_USDTUSD_1h_cleaned.csv'
# #    './source_data/TAIFEX_DLY_TXF1!, 60.csv'
]
def find_date_col(df):
    for col in df.columns:
        if 'date' in col.lower():
            return col
    return df.columns[0]


def get_prefix(file):
    if 'BTC' in file: return 'BTCUSDT'
    if 'ETH' in file: return 'ETHUSDT'
#    if 'DAI' in file: return 'DAIUSD'
    if 'USDCUSD' in file: return 'USDCUSD'
    if 'USDTUSD' in file: return 'USDTUSD'
    if 'ES1' in file: return 'ES1'
    if 'NQ1' in file: return 'NQ1'
    if 'YM1' in file: return 'YM1'
    if 'VX1' in file: return 'VIX'
    if 'ZN1' in file: return 'ZN1'
    if 'ZT1' in file: return 'ZT1'
    if 'ZF1' in file: return 'ZF1'
    if 'ZB1' in file: return 'ZB1'
    if 'TN1' in file: return 'TN1'
    if 'SR3' in file: return 'SR3'
#    if 'US10Y' in file: return 'US10Y'
    if 'GC1' in file: return 'GC1'
#    if 'DAI' in file: return 'DAIUSD'
    if 'USDCUSD' in file: return 'USDCUSD'
    if 'USDTUSD' in file: return 'USDTUSD'
#    if 'TXF1' in file: return 'TXF1'
    # fallback
    return os.path.splitext(os.path.basename(file))[0]

# 你之後再呼叫 get_prefix(file) 才不會出錯
def impute_and_flag(df, start, end, freq, asset_prefix):
    # """
    # df: 單一資產的 DataFrame（已經把日期欄改名為 'DATE'，其他欄已加上資產前綴）
    # start, end: datetime-like，可用你上面定義的 start/end
    # freq: 'D' 或 'H'
    # asset_prefix: 例如 'ES1'、'BTCUSDT'、'ZN1'…（用來命名 is_trading 欄位）
    # """
    # 1) 整理時間欄位並排序
    df = df.copy()
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
    df = df.dropna(subset=['DATE']).sort_values('DATE')
    df = df.loc[(df['DATE'] >= start) & (df['DATE'] <= end)]

    # 2) 建立完整索引並 reindex
    full_index = pd.date_range(start=start, end=end, freq=freq)
    df = df.set_index('DATE')
    original_idx = df.index

    df = df.reindex(full_index)

    # 3) is_trading 欄：原始有列=1；補出的=0
    flag_col = f"{asset_prefix}_is_trading"
    df[flag_col] = 0
    df.loc[original_idx.intersection(df.index), flag_col] = 1

    # 4) 針對欄位類型補值
    #   - OHLC → ffill
    #   - VOLUME / VOLUME_BASE → 0.0
    #   - 其他 → ffill
    for col in df.columns:
        if col == flag_col:
            continue
        # 注意：你的欄位已經含資產前綴，因此用 endswith 判斷後綴
        if col.endswith(('_OPEN', '_HIGH', '_LOW', '_CLOSE')):
            df[col] = df[col].ffill()
            # 如果整列都還是 NaN，就填 0
            df[col] = df[col].fillna(0.0)
        elif col.endswith(('_VOLUME', '_VOLUME_BASE')):
            df[col] = df[col].fillna(0.0)
        else:
            df[col] = df[col].ffill()
            df[col] = df[col].fillna(0.0)
    # 5) 把索引還原為 DATE 欄位（維持你既有慣例）
    df = df.reset_index().rename(columns={'index': 'DATE'})

    return df

def read_and_clean(file):
    df = pd.read_csv(file)
    date_col = find_date_col(df)
    tried = False
    for fmt in ['%Y-%m-%d %H:%M:%S', '%Y/%m/%d %H:%M', '%Y-%m-%d %H:%M', '%Y/%m/%d %H:%M:%S']:
        try:
            df[date_col] = pd.to_datetime(df[date_col], format=fmt, errors='raise')
            tried = True
            break
        except Exception:
            continue
    if not tried:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    # 強制 floor 到小時，避免交集失敗
    df[date_col] = df[date_col].dt.floor('h')
    df[date_col] = df[date_col].dt.tz_localize(None)
    df = df.rename(columns={date_col: 'DATE'})
    return df

dfs_day = []
for file in csv_files_day:
    df1 = pd.read_csv(os.path.join(folder, file))
    date_col1 = find_date_col(df1)
    df1[date_col1] = pd.to_datetime(df1[date_col1], errors='coerce')
    df1 = df1.dropna(subset=[date_col1])
    prefix = get_prefix(file)
    # 將所有非日期欄位加前綴
    rename = {col: f"{prefix}_{col.upper()}" for col in df1.columns if col != date_col1}
    df1 = df1.rename(columns=rename)
    df1 = df1.rename(columns={date_col1: 'DATE'})

    # 🔽🔽🔽【新增】日頻補值 + 旗標（freq='D'）
    df1 = impute_and_flag(df1, start, end, freq='D', asset_prefix=prefix)

    dfs_day.append(df1)

dfs_hour = []
for file in csv_files_hour:
    df2 = read_and_clean(file)
    date_col2 = find_date_col(df2)
    df2[date_col2] = pd.to_datetime(df2[date_col2], errors='coerce')
    df2 = df2.dropna(subset=[date_col2])
    prefix = get_prefix(file)
    # 將所有非日期欄位加前綴
    rename = {col: f"{prefix}_{col.upper()}" for col in df2.columns if col != date_col2}
    df2 = df2.rename(columns=rename)
    df2 = df2.rename(columns={date_col2: 'DATE'})

    # 🔽🔽🔽【新增】小時補值 + 旗標（freq='H'）
    df2 = impute_and_flag(df2, start, end, freq='h', asset_prefix=prefix)

    dfs_hour.append(df2)

# print(f"dfs_hour List")
# for i, df in enumerate(dfs_hour):
#     print(f"---- dfs_hour[{i}] ----")
#     # print(f"Shape: {df.shape}")
#     print("Columns:", df.columns.tolist())
#     #print(df.head(), "\n")
# print(f"\ndfs_day List")
# for i, df in enumerate(dfs_day):
#     print(f"---- dfs_day[{i}] ----")
#     # print(f"Shape: {df.shape}")
#     print("Columns:", df.columns.tolist())
#     #print(df.head(), "\n")

# 1. prefix → group 对照表，一定要把 list_prefixes 里所有的都加进来
GROUP_MAP = {
    'crypto': ['BTCUSDT','ETHUSDT','USDCUSD','USDTUSD'],
    'stock':  ['ES1','NQ1','YM1','VIX'],
    'bonds':  ['US10Y','ZN1','ZF1','ZT1','ZB1','TN1','SR3'],
    'others':  ['GC1']
}

def classify_dfs(dfs):
    groups = { g: [] for g in GROUP_MAP }
    for df in dfs:
        prefix = get_prefix(df.columns[1].split('_',1)[0])
        for grp, prefixes in GROUP_MAP.items():
            if prefix in prefixes:
                groups[grp].append(df)
                break
        else:
            raise AssertionError(f"前缀 {prefix} 没分到组，请补充到 GROUP_MAP")
    return groups



day_groups  = classify_dfs(dfs_day)
hour_groups = classify_dfs(dfs_hour)

for name, lst in day_groups.items():
    print(f"Day   / {name:6s}: {len(lst)} 檔")
for name, lst in hour_groups.items():
    print(f"Hour  / {name:6s}: {len(lst)} 檔")



# 假設前面已經有 day_groups, hour_groups 兩個 dict:
# { 'crypto':[df1,df2,…], 'stock':[…], 'bonds':[…] }
merged = {}
for freq, groups in (('day',   day_groups),
                     ('hour', hour_groups)):
    for name, lst in groups.items():
        # 用 reduce+merge 做交集
        key = f"df_merged_{freq}_{name}"
        merged[key] = reduce(
            lambda left, right: pd.merge(left, right, on='DATE', how='inner'),
            lst
        )

# merged 裡就會有
# merged['df_merged_day_crypto']
# merged['df_merged_day_stock']
# merged['df_merged_day_bonds']
# merged['df_merged_hour_crypto']
# merged['df_merged_hour_stock']
# merged['df_merged_hour_bonds']


# 你想要的 prefix 順序（不在裡面的 prefix 則跳過）
preferred_order = [
    'BTCUSDT', 'ETHUSDT','USDCUSD', 'USDTUSD',
    'ES1', 'NQ1', 'YM1','VIX',
    'SR3', 'ZT1', 'ZF1', 'TN1', 'ZN1','ZB1',
    'US10Y', 'GC1'
]

# 確保 output 目錄存在
out_dir = "./filtered_output/row_group/"
os.makedirs(out_dir, exist_ok=True)

for name, df in merged.items():
    # name 會長得像 "df_merged_hour_crypto"、"df_merged_day_stock"…
    # 我們把它拆成 freq + group，然後打入檔名
    _, _, freq, group = name.split("_")
    out_path = os.path.join(out_dir, f"{group}_{freq}_aligned.csv")

    # 把 DATE 放最前面
    cols = ["DATE"]

    # 照你的 preferred_order、以及 [CLOSE, VOLUME, OPEN, HIGH, LOW] 的順序挑欄位
    for suffix in ["CLOSE", "VOLUME", "OPEN", "HIGH", "LOW", "is_trading"]:
        for prefix in preferred_order:
            col = f"{prefix}_{suffix}"
            if col in df.columns:
                cols.append(col)

    # 重排欄位
    df_out = df[cols]

    # 寫 CSV
    df_out.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"✅ 輸出完成：{out_path}")



Day   / crypto: 4 檔
Day   / stock : 4 檔
Day   / bonds : 6 檔
Day   / others: 1 檔
Hour  / crypto: 4 檔
Hour  / stock : 4 檔
Hour  / bonds : 6 檔
Hour  / others: 1 檔
✅ 輸出完成：./filtered_output/row_group/crypto_day_aligned.csv
✅ 輸出完成：./filtered_output/row_group/stock_day_aligned.csv
✅ 輸出完成：./filtered_output/row_group/bonds_day_aligned.csv
✅ 輸出完成：./filtered_output/row_group/others_day_aligned.csv
✅ 輸出完成：./filtered_output/row_group/crypto_hour_aligned.csv
✅ 輸出完成：./filtered_output/row_group/stock_hour_aligned.csv
✅ 輸出完成：./filtered_output/row_group/bonds_hour_aligned.csv
✅ 輸出完成：./filtered_output/row_group/others_hour_aligned.csv
